## Create a Site-to-Site Virtual Private Network in Azure

* * *

This notebook uses [PowerShell Az.Network cmdlets](https://docs.microsoft.com/en-us/powershell/module/az.network/ "https://docs.microsoft.com/en-us/powershell/module/az.network/") to create a S2S VPN gateway connection from an on-premises network to an Azure Virtual Network (VPN).

![](https://docs.microsoft.com/en-us/azure/vpn-gateway/media/vpn-gateway-tutorial-vpnconnection-powershell/site-to-site-diagram.png)

Use these parameters to create the environment or to better understand the examples in this notebook.

| Line # | Name | Example | Description |
| --- | --- | --- | --- |
| 1 | **Subscription** | "" | Name or ID (guid) of Azure Subscription to setup S2S VPN in. |
| 2 | **ResourceGroup** | "TestRG1" | Name of new or existing resource group (RG). An Azure Resource Group is a collection of Azure resources that share the same permissions, policies, etc. In this case, the Resource Group for the virtual network is specified.  |
| 3 | **VnetName** | "VNet1" | Alphanumeric value represents the name of the Azure resource to create. |
| 4 | **Location** | "East US" | Value representing the region or location of the RG. See [Azure Geographies](https://azure.microsoft.com/en-us/global-infrastructure/geographies/ "https://azure.microsoft.com/en-us/global-infrastructure/geographies/") for more information. |
| 5 | **AddressSpace** | "10.1.0.0/16" | Defines a range of IP prefixes in the remote subnet. |
| 6 | **SubnetName** | "Frontend" |  |
| 7 | **Subnet** | "10.1.0.0/24" |  |
| 8 | **GatewaySubnet** | "10.1.255.0/27" | Specify the number of IP addresses that the subnet contains. The number of IP addresses needed depends on the VPN gateway configuration created. Some configurations require more IP addresses than others. It is recommended to create a gateway subnet that uses a /27 or /28. |
| 9 | **LocalNetworkGatewayName** | "Site1" |  |
| 10 | **LNGPublicIP** | "192.168.29.46" |  |
| 11 | **LocalAddressPrefix** | "10.101.0.0/24, 10.101.1.0/24" |  |
| 12 | **GatewayName** | "GatewaySubnet" |  |
| 13 | **PublicIP** | "VNet1GWPIP" |  |
| 14 | **GatewayIPConfig** | "gwipconfig1" |  |
| 15 | **VPNType** | "RouteBased" |  |
| 16 | **GatewayType** | "Vpn" |  |
| 17 | **ConnectionName** | "VNet1toSite1" |  |

In [ ]:
$Subscription               = ""
$ResourceGroup              = ""
$VnetName                   = ""
$Location                   = ""
$AddressSpace               = ""
$SubnetName                 = ""
$Subnet                     = ""
$GatewaySubnet              = ""
$LocalNetworkGatewayName    = ""
$LNGPublicIP                = ""
$LocalAddressPrefixes       = ""
$GatewayName                = ""
$PublicIP                   = ""
$GatewayIPConfig            = ""
$VPNType                    = ""
$GatewayType                = ""
$ConnectionName             = ""

There are a handful of steps to setup a resource group and configure the VPN:

1.  Create or specify a resource group
2.  Create a virtual network
3.  Create a subnet configuration
4.  Set the subnet configuration for the virtual network
5.  Add a gateway subnet
6.  Set the subnet configuration for the virtual network
7.  Request a public IP address
8.  Create the gateway IP address configuration
9.  Create the VPN gateway
10.  Create the local network gateway
11.  Create the VPN connection

## Create a resource group

In [ ]:
New-AzResourceGroup -Name $ResourceGroup -Location $Location

## Create a virtual network

This example creates a virtual network and a gateway subnet. If the virtual network is already present then gateway subnet is needed, see [To add a gateway subnet to a virtual network which is already created](https://docs.microsoft.com/en-us/azure/vpn-gateway/vpn-gateway-create-site-to-site-rm-powershell#gatewaysubnet "https://docs.microsoft.com/en-us/azure/vpn-gateway/vpn-gateway-create-site-to-site-rm-powershell#gatewaysubnet"). When creating a virtual network, make sure that the address spaces don't overlap any of the address spaces that are in on-premises network.

In [ ]:
$virtualNetwork = New-AzVirtualNetwork `
  -ResourceGroupName $ResourceGroup `
  -Location $Location `
  -Name $VnetName `
  -AddressPrefix $AddressSpace

## Create a subnet configuration

In [ ]:
$subnetConfig = Add-AzVirtualNetworkSubnetConfig `
  -Name $SubnetName `
  -AddressPrefix $Subnet `
  -VirtualNetwork $virtualNetwork

## Set the subnet configuration for the virtual network

In [ ]:
$virtualNetwork | Set-AzVirtualNetwork

## Add a gateway subnet
The virtual network gateway uses specific subnet called the gateway subnet. The gateway subnet is part of the virtual network IP address range that you specify when configuring your virtual network. It contains the IP addresses that the virtual network gateway resources and services use. The subnet must be named 'GatewaySubnet' in order for Azure to deploy the gateway resources. You can't specify a different subnet to deploy the gateway resources to. If you don't have a subnet named 'GatewaySubnet', when you create your VPN gateway, it will fail.

In [ ]:
$vnet = Get-AzVirtualNetwork -ResourceGroupName $ResourceGroup -Name $VnetName
Add-AzVirtualNetworkSubnetConfig -Name $GatewayName -AddressPrefix $GatewaySubnet -VirtualNetwork $vnet

## Set the subnet configuration for the virtual network

In [ ]:
$vnet | Set-AzVirtualNetwork

## Request a public IP address
A VPN gateway must have a Public IP address. It first requests the IP address resource, and then refer to it when creating virtual network gateway. The IP address is dynamically assigned to the resource when the VPN gateway is created.

VPN Gateway currently only supports Dynamic Public IP address allocation. It cannot request a Static Public IP address assignment. However, this does not mean that the IP address will change after it has been assigned to created VPN gateway. The only time the Public IP address changes is when the gateway is deleted and re-created. It doesn't change across resizing, resetting, or other internal maintenance/upgrades of created VPN gateway.

In [ ]:
$gwpip= New-AzPublicIpAddress -Name $PublicIP -ResourceGroupName $ResourceGroup -Location $Location `
 -AllocationMethod Dynamic

## Create the gateway IP address configuration
The gateway configuration defines the subnet (the 'GatewaySubnet') and the public IP address to use. Use the following example to create gateway configuration:

In [ ]:
$vnet = Get-AzVirtualNetwork -Name $VnetName -ResourceGroupName $ResourceGroup
$subnet = Get-AzVirtualNetworkSubnetConfig -Name $GatewayName -VirtualNetwork $vnet
$gwipconfig = New-AzVirtualNetworkGatewayIpConfig -Name $GatewayIPConfig -SubnetId $subnet.Id -PublicIpAddressId $gwpip.Id

## Create a gateway
A gateway can take 45 minutes or more to create. Once the gateway creation has completed, you can create a connection between your VPN and another VNet. Or create a connection between your VPN and an on-premises location. Create a gateway using the _New-AzVirtualNetworkGateway_ cmdlet.

In [ ]:
New-AzVirtualNetworkGateway -Name $GatewayName -ResourceGroupName $ResourceGroup `
 -Location $Location -IpConfigurations $gwipconfig -GatewayType $GatewayType `
 -VpnType $VPNType -GatewaySku VpnGw1

## Create the local network gateway
The local network gateway (LNG) typically refers to on-premises location. It is not the same as a virtual network gateway. Give the site a name by which Azure can refer to it, then specify the IP address of the on-premises VPN device to the connection to be created. Also specify the IP address prefixes that will be routed through the VPN gateway to the VPN device. The address prefixes specified are the prefixes located on on-premises network. If on-premises network changes, it can easily update the prefixes.

In [ ]:
New-AzLocalNetworkGateway -Name $LocalNetworkGatewayName -ResourceGroupName $ResourceGroup `
 -Location $Location -GatewayIpAddress '192.168.29.46' -AddressPrefix @('10.101.0.0/24','10.101.1.0/24')

## Create the VPN connection
Create the Site-to-Site VPN connection between virtual network gateway and VPN device. Be sure to replace the values. The shared key must match the value used for VPN device configuration. Notice that the '-ConnectionType' for Site-to-Site is IPsec.

In [ ]:
$gateway1 = Get-AzVirtualNetworkGateway -Name $GatewayName -ResourceGroupName $ResourceGroup
$local = Get-AzLocalNetworkGateway -Name $LocalNetworkGatewayName -ResourceGroupName $ResourceGroup
New-AzVirtualNetworkGatewayConnection -Name $ConnectionName -ResourceGroupName $ResourceGroup `
 -Location $Location -VirtualNetworkGateway1 $gateway1 -LocalNetworkGateway2 $local `
 -ConnectionType IPsec -RoutingWeight 10 -SharedKey 'abc123'

## Verify the VPN connection
Connection can be verified if it is succeeded by using the _Get-AzVirtualNetworkGatewayConnection_ cmdlet, with or without '-Debug'.

In [ ]:
Get-AzVirtualNetworkGatewayConnection -Name $ConnectionName -ResourceGroupName $ResourceGroup